In [7]:
# ==1.2.2

In [4]:
 # %pip install robin_stocks openai==1.2.2 yfinance requests_oauthlib twilio boto3 --upgrade 

In [5]:
import robin_stocks.robinhood as r
import requests
import json
import pandas as pd
import datetime
from dateutil import parser
from dateutil.relativedelta import relativedelta
from dateutil.tz import tzlocal
from datetime import timedelta
import pytz
import boto3
from tqdm.auto import tqdm
from io import StringIO, BytesIO

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [6]:
from news2roi import News2ROI

In [7]:
s3 = boto3.client('s3')

In [8]:
bucket_name = 'jvalansi-notebooks-data'

In [6]:
# s3.upload_file("/root/.aws/secrets.json", bucket_name, ".aws.json")

In [7]:
# s3.upload_file("/root/.rapidapi.json", bucket_name, ".rapidapi.json")

In [8]:
# s3.upload_file("/root/.newsapi.json", bucket_name, ".newsapi.json")

In [9]:
data = s3.get_object(Bucket=bucket_name, Key=".aws.json")
j = json.loads(data['Body'].read().decode('utf8'))
openai_key = j['OPENAI_KEY']
data = s3.get_object(Bucket=bucket_name, Key=".rapidapi.json")
# data = s3.get_object(Bucket=bucket_name, Key=".newsapi.json")
j = json.loads(data['Body'].read().decode('utf8'))
news_key = j["key"]


In [9]:
# s3.upload_file("/root/.robinhood.json", bucket_name, ".robinhood.json")

In [10]:
data = s3.get_object(Bucket=bucket_name, Key=".robinhood.json")
trade_cred = json.loads(data['Body'].read().decode('utf8'))

In [11]:
data = s3.get_object(Bucket=bucket_name, Key=".twilio.json")
notify_cred = json.loads(data['Body'].read().decode('utf8'))

In [12]:
n2r = News2ROI(openai_key, news_key, trade_cred, notify_cred)

In [13]:
date = datetime.datetime.today().strftime('%Y-%m-%d')

In [ ]:
# date = "2024-01-26"

In [ ]:
# date = "2024-01-16"

In [14]:
# get new news articles
news = n2r.get_news(date=date)

In [15]:
len(news)

20

In [16]:
articles = news #['articles']

In [17]:
articles = [article for article in articles if News2ROI.contains_words(article, words={'United States',"US"})]

In [18]:
len(articles)

9

In [19]:
sorted([article['publishedAt']['date'] for article in articles])

['2024-02-07 00:01:32.000000',
 '2024-02-07 00:02:04.000000',
 '2024-02-07 00:17:25.000000',
 '2024-02-07 00:28:26.000000',
 '2024-02-07 00:31:56.000000',
 '2024-02-07 00:48:04.000000',
 '2024-02-07 00:49:42.000000',
 '2024-02-07 03:00:33.000000',
 '2024-02-07 03:11:47.000000']

In [20]:
articles_ = [article for article in articles if n2r.is_recent(article['publishedAt']['date'], delta=datetime.timedelta(hours=10))]

In [21]:
len(articles_)

9

In [23]:
# get sentiment
res = []


In [24]:
for article in tqdm(articles):
    res += [n2r.analyse_article(article)]
    

  0%|          | 0/9 [00:00<?, ?it/s]

In [28]:
candidates = News2ROI.get_candidates(res, threshold=4)

In [29]:
candidates

,stock,ticker,exchange,reasoning,action,date,data
6,Colgate-Palmolive Company,CL,New York Stock Exchange,The lawsuit regarding misleading recyclability...,4,2024-02-07 00:28:26.000000,Colgate-Palmolive must face lawsuit over tooth...


In [36]:
# ticker = low_sentiment.iloc[0]['ticker']

In [30]:
candidate = candidates.iloc[0].copy(deep=True)

In [31]:
ticker = candidate['ticker']

In [14]:
ticker = 'aapl'

In [15]:
# get options


In [16]:
# expirationDate = the next friday

In [32]:
option_data = n2r.get_option_data(ticker)

ERROR: There was an issue loading pickle file. Authentication may be expired - logging in normally.
Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data 

In [33]:
len(option_data)

39

In [34]:
option_df = News2ROI.parse_option_data(option_data)

In [125]:
# option_df.plot(x='strike_price', y='adjusted_mark_price')

In [124]:
# option_df.plot(x='strike_price', y='break_even_price')

In [122]:
# find lowest option price with highest break even price
# usually the current stock price
# example stock costs 100
# option to put (sell) in a week for 100 costs 1$
# if the stock costs 99 in a week, I break even, anything below I earn
# e.g if the stock costs 98 in a week, I sell at 100, I get 2$, but it cost 1$, I made 1$

# if I buy the option to sell in 150, it costs 49.85
# if the stock costs 100 in a week, I sell at 150, I made 50, but it cost me 49.85, I made 0.15$

# if I buy the option to sell in 90, it costs 0.02
# if the stock costs 100 in a week, I sell in 90, I lost 10$

# options are at a multiples of 100s
# so if the stock costs 100
# and costs 4 to sell at 104
# if the stock goes to 101, it'll cost 3, so I could sell at 3$ and lose 100
# I could lose everythin
# If it goes to 99, it'll cost 5, and I could sell at 5, and gain 100
# I could 

In [74]:
# I want to maximize ROI
# I need to check which one maximizes for the expected value of 0.2% drop

In [35]:
row = option_df[option_df['normalized_gain']==option_df['normalized_gain'].max()].iloc[0].to_dict()

In [42]:
# candidate = pd.Series()

In [109]:
candidate['strike_price'] = row['strike_price']

candidate['buy_price'] = row['bid_price']

candidate['stock_buy_price'] = row['current_price']

In [28]:
# when the target price hits, the option prices should shift with the target price
# say current price is 100

In [ ]:
# maybe easier to calculate - each price shifts one up

In [ ]:
# place option for the day
# r.order_buy_option_limit

In [ ]:
# place reminder to sell tomorrow


In [ ]:
# sell previous reminders


In [139]:
# df = pd.DataFrame(candidate).T

In [39]:
def load_candidates(bucket_name):
    s3 = boto3.client('s3')
    data = s3.get_object(Bucket=bucket_name, Key="candidates.tsv")
    data = data['Body'].read().decode('utf8')
    df = pd.read_csv(StringIO(data), sep='\t', index_col=0)
    return df

In [153]:
df = load_candidates(bucket_name)

In [154]:
df

,stock,ticker,exchange,reasoning,action,date,data,strike_price,buy_price,stock_buy_price,sell_price,stock_sell_price,roi
0,Vista Outdoor Inc.,VSTO,NYSE,The news about California's ammunition backgro...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
1,Vista Outdoor Inc.,VSTO,NYSE,As a manufacturer and seller of ammunition and...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
2,Vista Outdoor Inc.,VSTO,NYSE,The news about California's ammunition backgro...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
3,Colgate-Palmolive Company,CL,NYSE,The news of Colgate-Palmolive facing a lawsuit...,3.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,0.25,84.35,1.666667
4,Colgate-Palmolive Company,CL,New York Stock Exchange (NYSE),The lawsuit against Colgate-Palmolive may have...,3.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,NaN,NaN,NaN
-1,Colgate-Palmolive Company,CL,New York Stock Exchange,The lawsuit regarding misleading recyclability...,4.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,NaN,NaN,NaN


In [160]:
df['date'] = pd.to_datetime(df['date'])

In [81]:
now = datetime.datetime.now()

In [137]:
to_sell = df[(now - datetime.timedelta(hours=24) < df['date']) & (df['date'] < now - datetime.timedelta(hours=5))].iloc[0]

In [138]:
to_sell

stock                                       Colgate-Palmolive Company
ticker                                                             CL
exchange                                                         NYSE
reasoning           The news of Colgate-Palmolive facing a lawsuit...
action                                                            3.0
date                                              2024-02-07 00:28:26
data                Colgate-Palmolive must face lawsuit over tooth...
strike_price                                                     83.0
buy_price                                                        0.15
stock_buy_price                                                 84.35
sell_price                                                       0.25
stock_sell_price                                                84.35
Name: 3, dtype: object

In [103]:
sell_options = n2r.get_option_data(to_sell['ticker'])

Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data Loading Market Data 

In [105]:
sell_options = News2ROI.parse_option_data(sell_options)

In [111]:
row = sell_options.set_index('strike_price').loc[to_sell['strike_price']]

In [133]:
df = df.rename({"bid_price": "buy_price"}, axis=1)

In [135]:
df = df.rename({"current_price": "stock_buy_price"}, axis=1)

In [141]:
df.loc[to_sell.name, 'sell_price'] = row['ask_price']

df.loc[to_sell.name, 'stock_sell_price'] = row['current_price']

df.loc[to_sell.name, 'roi'] = row['ask_price']/to_sell['buy_price']

In [142]:
df

,stock,ticker,exchange,reasoning,action,date,data,strike_price,buy_price,stock_buy_price,sell_price,stock_sell_price,roi
0,Vista Outdoor Inc.,VSTO,NYSE,The news about California's ammunition backgro...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
1,Vista Outdoor Inc.,VSTO,NYSE,As a manufacturer and seller of ammunition and...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
2,Vista Outdoor Inc.,VSTO,NYSE,The news about California's ammunition backgro...,5.0,2024-02-06 03:08:50,California ammunition background check law can...,27.5,0.05,29.95,NaN,NaN,NaN
3,Colgate-Palmolive Company,CL,NYSE,The news of Colgate-Palmolive facing a lawsuit...,3.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,0.25,84.35,1.666667
4,Colgate-Palmolive Company,CL,New York Stock Exchange (NYSE),The lawsuit against Colgate-Palmolive may have...,3.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,NaN,NaN,NaN
-1,Colgate-Palmolive Company,CL,New York Stock Exchange,The lawsuit regarding misleading recyclability...,4.0,2024-02-07 00:28:26,Colgate-Palmolive must face lawsuit over tooth...,83.0,0.15,84.35,NaN,NaN,NaN


In [ ]:
store_candidate(

In [113]:
to_sell

stock                                       Colgate-Palmolive Company
ticker                                                             CL
exchange                                                         NYSE
reasoning           The news of Colgate-Palmolive facing a lawsuit...
action                                                            3.0
date                                              2024-02-07 00:28:26
data                Colgate-Palmolive must face lawsuit over tooth...
strike_price                                                     83.0
bid_price                                                        0.15
current_price                                                   84.35
sell_price                                                       0.25
stock_sell_price                                                84.35
Name: 3, dtype: object

In [ ]:
df.loc[-1] = candidate
df = df.reset_index(drop=True)


In [151]:
def store_candidates(bucket_name, df):
    df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')
    s_buf = BytesIO()
    df.to_csv(s_buf, sep='\t')
    s_buf.seek(0)
    s3.upload_fileobj(s_buf, bucket_name, "candidates.tsv")


In [161]:
store_candidates(bucket_name, df)

In [45]:
print(candidate.to_json(indent=2))

{
  "stock":"Colgate-Palmolive Company",
  "ticker":"CL",
  "exchange":"New York Stock Exchange",
  "reasoning":"The lawsuit regarding misleading recyclability claims may negatively impact Colgate-Palmolive's brand image and consumer trust. It could also lead to financial penalties or to increased costs if the company is required to adjust its packaging to be more environmentally friendly. There could be a short-term drop in stock price due to these factors, as investors react to potential legal fees, settlements, and the potential cost of rebranding or repackaging products. Additionally, ESG (environmental, social, and governance) considerations are increasingly important to investors, and negative news in this arena might affect investment decisions.",
  "action":4,
  "date":"2024-02-07 00:28:26.000000",
  "data":"Colgate-Palmolive must face lawsuit over toothpaste tube recycling claims - Colgate-Palmolive must face a lawsuit alleging it misleads consumers by claiming its plastic too

In [146]:
# j = {'account_sid': 'AC417be496c2958add9742abcc83840cb7',
#      'auth_token': '32e6cee11411c6037525b250c973eb2f',
#      'from': '+18662593587',
#      'to': '+14159002444'}


In [110]:
# with open('/root/.twilio.json', 'w') as f:
#     f.write(json.dumps(j))

In [111]:
# s3.upload_file("/root/.twilio.json", bucket_name, ".twilio.json")

In [46]:
n2r.notify(candidate)

'SMd2edc028d742480b4ec41eed8b151b26'